In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the saved model and tokenizer
MODEL_NAME = "models/Model 3 - Transfer Learning with BERT/bert_sentiment_model"  # Trained BERT model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


/home/k3v1n/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [2]:
def predict_sentiment(review, model, tokenizer, max_length=128):
    """
    Predict sentiment for a single review.
    
    Args:
        review (str): The input review as a string.
        model: The fine-tuned BERT model.
        tokenizer: The tokenizer corresponding to the BERT model.
        max_length (int): The maximum token length for the review.
    
    Returns:
        dict: The predicted label (0 for negative, 1 for positive) and probabilities.
    """
    # Tokenize the review
    inputs = tokenizer(
        review,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Move input tensors to the device
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()
    
    # Map label to sentiment
    sentiment = "Positive" if predicted_label == 1 else "Negative"
    return {"sentiment": sentiment, "probabilities": probabilities.cpu().numpy()}


In [3]:
# Sample review
sample_review = "I hated this product! I am never buying this again"

# Predict sentiment
result = predict_sentiment(sample_review, model, tokenizer)
print(f"Sentiment: {result['sentiment']}")
print(f"Probabilities: {result['probabilities']}")

Sentiment: Negative
Probabilities: [[0.9978563  0.00214363]]
